In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from flair.models import TextClassifier
from flair.data import Sentence
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def sentiment(text):
    global analyser
    try:
        sentence = Sentence(text)
        analyser.predict(sentence)
    except:
        analyser = TextClassifier.load('en-sentiment')
        sentence = Sentence(text)
        analyser.predict(sentence)
    finally:
        return str(sentence.labels[0])[:-9]

def news(companies):
        if companies.split() != 1:
            companies = [i.upper() for i in companies.strip().split()]
        else:
            companies = [companies.upper()]
        finviz_url = 'https://finviz.com/quote.ashx?t='
        news_tables = {}
        for company in companies:
            url = finviz_url + company
            req = Request(url=url, headers={'user-agent': 'app'})
            response = urlopen(req)
            html = BeautifulSoup(response, features='html.parser')
            news_table = html.find(id='news-table')
            news_tables[company] = news_table

        parsed_data = []
        for company, news_table in news_tables.items():
            for row in news_table.findAll('tr'):
                title = row.a.text
                date_data = row.td.text.split(' ')
                if len(date_data) != 1:
                    date = date_data[0]
                parsed_data.append([company, date, title])

        df = pd.DataFrame(parsed_data, columns=['Company', 'Date', 'Headline'])
        df['Date'] = pd.to_datetime(df.Date).dt.date
        df['Sentiment'] = df['Headline'].map(sentiment)
        result = pd.DataFrame(columns=df.columns)
        for i in df['Company'].unique():
            result = pd.concat([result, df[df['Company'] == i].iloc[:10,:]])           
        result = result.reset_index(drop=True)
        return result

In [3]:
df = news('amd fb')
df

2021-09-09 08:59:16,242 loading file C:\Users\nayak\.flair\models\sentiment-en-mix-distillbert_4.pt


,Company,Date,Headline,Sentiment
0,AMD,2021-09-08,Garmin (GRMN) Boosts Marine Segment With Surro...,POSITIVE
1,AMD,2021-09-08,Is Invesco Dynamic Semiconductors ETF (PSI) a ...,POSITIVE
2,AMD,2021-09-07,"Have $2,000? 2 Growth Stocks to Buy and Hold f...",POSITIVE
3,AMD,2021-09-07,"Strategist talks Federal Reserve, bonds, and T...",POSITIVE
4,AMD,2021-09-07,4 Top-Ranked Semi Stocks to Buy as Global Sale...,POSITIVE
5,AMD,2021-09-07,"Stock Market News for Sep 7, 2021",NEGATIVE
6,AMD,2021-09-07,"Is Nvidia Stock A Buy After Strong Earnings, G...",NEGATIVE
7,AMD,2021-09-06,10 Trending Stocks on Reddit,NEGATIVE
8,AMD,2021-09-05,2 Ways to Profit From China's Growth (Without ...,NEGATIVE
9,AMD,2021-09-04,Nvidia Trounces AMD Once Again: Here's Why Tha...,NEGATIVE
